In [1]:
import re
from collections import Counter

## 读取原始数据

In [2]:
names = ['train', 'val', 'test']
real_all = []
synt_all = []
for name in names:
    real_file = f'./dataset/{name}_real.txt'
    with open(real_file, 'r') as f:
        real = []
        for line in f:
            if line.strip().endswith('_back'):
                continue
            real.append(line.strip())
    real_all += real
     
    synt_file = f'./dataset/{name}_synt.txt'
    with open(synt_file, 'r') as f:
        synt = []
        for line in f:
            if line.strip().endswith('_back'):
                continue
            line = line.strip().replace('_front', '')
            synt.append(line)
    synt_all += synt

In [3]:
print(len(real_all))
print(len(set(real_all)))
print(len(synt_all))
print(len(set(synt_all)))

1043
1043
12550
12403


## 原始分布

In [4]:
reals = [line.split('_')[0] for line in set(real_all)]
Counter(reals)

Counter({'Links2': 105,
         'Mesh': 122,
         'Tuck': 167,
         'Miss': 65,
         'Hem': 134,
         'Move1': 114,
         'Move2': 138,
         'Cable1': 69,
         'Links1': 42,
         'Cable2': 71,
         'Full': 16})

In [5]:
synts = [line.split('_')[0] for line in set(synt_all)]
Counter(synts)

Counter({'Move2': 2858,
         'Miss': 1419,
         'Cable1': 1312,
         'Move1': 1522,
         'Hem': 2686,
         'Cable2': 829,
         'Links2': 981,
         'Mesh': 229,
         'Links1': 261,
         'Tuck': 303,
         'Full': 3})

In [6]:
synt_match = []
for line in set(synt_all):
    if line in set(real_all) or line+'_front' in set(real_all):
        synt_match.append(line)

In [7]:
len(synt_match)

988

In [8]:
synt_matchs = [line.split('_')[0] for line in set(synt_match)]
Counter(synt_matchs)

Counter({'Links2': 105,
         'Links1': 42,
         'Move1': 98,
         'Mesh': 122,
         'Miss': 65,
         'Move2': 114,
         'Cable1': 69,
         'Hem': 134,
         'Tuck': 167,
         'Cable2': 71,
         'Full': 1})

## 增加非real

In [9]:
residuals = dict()
for k, v in Counter(synt_matchs).items():
    if k == 'Full':
        continue
    resi = min(300 - v, Counter(synts)[k] - v)
    residuals[k] = resi
residuals

{'Links2': 195,
 'Links1': 219,
 'Move1': 202,
 'Mesh': 107,
 'Miss': 235,
 'Move2': 186,
 'Cable1': 231,
 'Hem': 166,
 'Tuck': 133,
 'Cable2': 229}

In [10]:
synt_tmp = [line for line in synt_match if not line.startswith('Full')]
for cat in residuals.keys():
    total = residuals[cat]
    i = 0
    for line in set(synt_all):
        if line not in synt_match and line.startswith(cat) and i < total:
            synt_tmp.append(line)
            i += 1

In [11]:
len(synt_tmp)

2890

In [12]:
synt_tmps = [line.split('_')[0] for line in set(synt_tmp)]
Counter(synt_tmps)

Counter({'Move1': 300,
         'Miss': 300,
         'Move2': 300,
         'Cable2': 300,
         'Mesh': 229,
         'Tuck': 300,
         'Cable1': 300,
         'Hem': 300,
         'Links2': 300,
         'Links1': 261})

## 凑到3000个

In [13]:
print(3000 - len(synt_tmp))

110


In [14]:
res2 = {
    'Cable1': 15,
    'Hem': 16,
    'Miss': 15,
    'Move2': 16,
    'Links2': 15,
    'Move1': 15,
    'Mesh': 0,
    'Cable2': 15,
    'Links1': 0,
    'Tuck': 3
}

In [15]:
synt_final = synt_tmp.copy()
for cat in res2.keys():
    total = res2[cat]
    i = 0
    for line in set(synt_all):
        if line not in synt_tmp and line.startswith(cat) and i < total:
            synt_final.append(line)
            i += 1
len(synt_final)

3000

In [32]:
real_final = []
for line in real_all:
    if not line.startswith('Full') and (line in synt_match or line.replace('_front', '') in synt_match):
        real_final.append(line)
len(real_final)

987

In [33]:
inst_tmp = [line.replace('_front', '') for line in real_final]
resi3 = [line for line in synt_final if not line in inst_tmp]
inst_final = real_final + resi3
len(inst_final)

3000